In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import json
import os

In [2]:
#importing, scaling reshaping and vectorizing the data set
((trainX, trainY), (testX, testY)) = cifar10.load_data()

#scaling data
trainX = trainX.astype('float')/255.0
testX = testX.astype('float')/255.0


#vectorizing the labels
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

In [3]:
#initializing the label names
labelNames = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 
             'frog', 'horse', 'ship', 'truck']

In [4]:
#initializing the model
chanDim= -1
def model():
    
    model = Sequential()
    #first layer of Conv2D
    model.add(Conv2D(32, (3,3), padding = 'same', input_shape = (32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(32, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    #second layer of Conv2D
    model.add(Conv2D(64, (3,3), padding = 'same', input_shape = (32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(64, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    #FC layet
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model

In [5]:
#initializing the optimizer and model
opt = SGD(lr = 0.01, decay = 0.01/40, momentum = 0.9, nesterov = True)
model = model()
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________

In [6]:
checkpoint = ModelCheckpoint('weights/cifar10_best_weights.hdf5', monitor = 'val_loss', 
                             mode = 'min', save_best_only = True, 
                             verbose = 1)
callbacks = [checkpoint]

H = model.fit(trainX, trainY, validation_data = (testX, testY), 
              batch_size = 64, epochs = 40, callbacks = callbacks, 
              verbose = 1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
49728/50000 [============================>.] - ETA: 0s - loss: 1.6350 - acc: 0.4524- ETA: 1s - loss: 1.6905 - a - ETA: 1s - loss: 1.668
Epoch 00001: val_loss improved from inf to 1.36887, saving model to weights/cifar10_best_weights.hdf5
50000/50000 [==============================] - 13s 264us/sample - loss: 1.6329 - acc: 0.4530 - val_loss: 1.3689 - val_acc: 0.5316
Epoch 2/40
49792/50000 [============================>.] - ETA: 0s - loss: 1.1733 - acc: 0.5896
Epoch 00002: val_loss improved from 1.36887 to 0.96748, saving model to weights/cifar10_best_weights.hdf5
50000/50000 [==============================] - 11s 217us/sample - loss: 1.1727 - acc: 0.5900 - val_loss: 0.9675 - val_acc: 0.6585
Epoch 3/40
49984/50000 [============================>.] - ETA: 0s - loss: 0.9745 - acc: 0.6551
Epoch 00003: val_loss improved from 0.96748 to 0.84864, saving model to weights/cifar10_best_weights.hdf5
50000/50000 [==========================

Epoch 27/40
49856/50000 [============================>.] - ETA: 0s - loss: 0.3311 - acc: 0.8810
Epoch 00027: val_loss did not improve from 0.54084
50000/50000 [==============================] - 12s 241us/sample - loss: 0.3314 - acc: 0.8809 - val_loss: 0.5470 - val_acc: 0.8240
Epoch 28/40
49856/50000 [============================>.] - ETA: 0s - loss: 0.3238 - acc: 0.8838
Epoch 00028: val_loss did not improve from 0.54084
50000/50000 [==============================] - 12s 231us/sample - loss: 0.3237 - acc: 0.8838 - val_loss: 0.5523 - val_acc: 0.8219
Epoch 29/40
49984/50000 [============================>.] - ETA: 0s - loss: 0.3219 - acc: 0.8829
Epoch 00029: val_loss did not improve from 0.54084
50000/50000 [==============================] - 11s 217us/sample - loss: 0.3219 - acc: 0.8829 - val_loss: 0.5544 - val_acc: 0.8199
Epoch 30/40
49856/50000 [============================>.] - ETA: 0s - loss: 0.3139 - acc: 0.8877
Epoch 00030: val_loss did not improve from 0.54084
50000/50000 [=========